# Getting the possible locations from the companies dataset

In [3]:
from pymongo import MongoClient
import pandas as pd


In [4]:
client = MongoClient()
db = client.get_database("companies")
comp = db.companies

### Considerations:
- We need to divide the offices dictionaries in its different offices using unwind so we have one single coordinate in each document.
- We should eliminate the null values.
- Our data should look like this in order to extract the locations:
```python
{
    "type": "Point",
    "coordinates": [125.6, 10.1] # LONG, LAT
}

```

In [3]:
res = comp.aggregate([
                      {"$unwind":"$offices" },
                      {"$match":{"offices.latitude":{"$ne":None}}},
                      {"$match":{"offices.longitude":{"$ne":None}}},
                      {"$project":{"_id":0}} 
                     ])

In [4]:
db.companies_unwinded.insert_many(res)

In [17]:
offices = db.companies_unwinded

In [18]:
res = offices.find( {"offices.country_code":"USA"})
len(list(res))

7600

In [47]:
number_of_offices = 7600

7600 of those unwinded companies are located in the USA, that means that **we have 7600 offices locations to choose.**

 We want our location to follow the geojson format, so let's convert it.

*This is the office attribute.*

In [7]:
res = offices.find( {}, {"offices":1})
list(res.limit(1))

[{'_id': ObjectId('5fb00ed5a020aa91fd9251a2'),
  'offices': {'description': '',
   'address1': '710 - 2nd Avenue',
   'address2': 'Suite 1100',
   'zip_code': '98104',
   'city': 'Seattle',
   'state_code': 'WA',
   'country_code': 'USA',
   'latitude': 47.603122,
   'longitude': -122.333253}}]

*This is the coord (geojson) and office attribute.*

In [11]:
res = offices.find({},{"offices":1})
for c in res:
    filt = {"_id":c["_id"]}
    coord = {
              "type": "Point",
              "coordinates": [c["offices"]["longitude"], c["offices"]["latitude"]]
            }
    update = {"$set":{"coord":coord}}
    offices.update_one(filt,update)

In [12]:
res = offices.find( {}, {"offices":1, "coord":1})

In [13]:
list(res.limit(1))

[{'_id': ObjectId('5fb00ed5a020aa91fd9251a2'),
  'offices': {'description': '',
   'address1': '710 - 2nd Avenue',
   'address2': 'Suite 1100',
   'zip_code': '98104',
   'city': 'Seattle',
   'state_code': 'WA',
   'country_code': 'USA',
   'latitude': 47.603122,
   'longitude': -122.333253},
  'coord': {'type': 'Point', 'coordinates': [-122.333253, 47.603122]}}]

**Now we can use this database to find companies near a certain point, list of points, polygon...**

[https://docs.mongodb.com/manual/reference/operator/query/near/](https://docs.mongodb.com/manual/reference/operator/query/near/)

We have found out that there are 7600, locations in the USA. Let's see where are they.

In [82]:
res = offices.find( {"offices.country_code":"USA"}, {"offices.state_code":1})

In [83]:
states = [res[i]['offices']['state_code'] for i in range(number_of_offices)]

## Offices by state

The amount of offices in California is much bigger that the amount of offices in the rest of the country. However, in a second position we have New York and, even though it has a lot less offices, it could also be a good location for our office.

In [84]:
df = pd.DataFrame(columns = ['states'])
df.states = states
df.states.value_counts().head(4)

CA    2968
NY     774
MA     451
TX     386
Name: states, dtype: int64

## Offices by state in the game_video category 

Let's study another parameter, in which state there are most companies in the game_video category so we can be close to out competitors and learn from them.

In [85]:
res = offices.find( {"offices.country_code":"USA", "category_code": "games_video"}, {"offices.state_code":1})
states = [res[i]['offices']['state_code'] for i in range(464)]

In [86]:
df = pd.DataFrame(columns = ['states'])
df.states = states
df.states.value_counts().head(4)

CA    220
NY     65
WA     23
MA     18
Name: states, dtype: int64

 # Our company will be located in California

### We have a lot more data to compare and it is the state with the higuest number of gaming companies. Let's save this information in a DataFrame.

If we select USA and CA, there are some points outside california that are also in the set, so I selected CA using the map in mongo  compass.

In [99]:
filter_q = {"coord.coordinates": {"$geoWithin": { "$geometry": { "type": "Polygon", "coordinates": [ [ [ -123.36526329703172, 38.411863094210894 ], [ -120.9940831325274, 34.66747439969875 ], [ -117.07724646425912, 32.27570902526987 ], [ -116.35711228118673, 32.67643502670626 ], [ -120.11586875309942, 39.54695688156478 ], [ -123.36526329703172, 38.411863094210894 ] ] ] }}}}
project_q = {"name":1, "coord":1,"offices.address1":1}
res = offices.find(filter_q, project_q)

In [98]:
len(list(res))

2925

In [100]:
name = []
address = []
long_lat = []
for i in range(2925):
    name.append(res[i]['name'])
    address.append(res[i]['offices']['address1'])
    long_lat.append(res[i]['coord']['coordinates'][::-1])

In [101]:
df = pd.DataFrame()
df['name'] = name
df['adress'] = address
df['long_lat'] = long_lat
df

,name,adress,long_lat
0,AdventNet,4900 Hopyard Rd.,"[37.692934, -121.904945]"
1,Zoho,4900 Hopyard Rd,"[37.692934, -121.904945]"
2,Digg,135 Mississippi St,"[37.764726, -122.394523]"
3,Facebook,1601 Willow Road,"[37.41605, -122.151801]"
4,Postini,"959 Skyway Road, Suite 200","[37.506885, -122.247573]"
...,...,...,...
2920,Purfresh,"1350 Willow Road,","[37.477008, -122.151599]"
2921,MyChances,,"[37.4484914, -122.1802812]"
2922,InSound Medical,39660 Eureka Drive,"[37.512038, -121.998513]"
2923,Intergy,11875 Dublin Boulevard,"[37.7019395, -121.9366984]"


# After the research we are going to locate our office in Los Angeles

In [8]:
offices = db.companies_unwinded
filter_q = {'offices.country_code': 'USA','offices.state_code': 'CA','offices.city': 'Los Angeles' }
project_q = {"name":1, "coord":1,"offices.address1":1}
res = offices.find(filter_q, project_q)

In [10]:
list(res)

[{'_id': ObjectId('5fb00ed5a020aa91fd9251b4'),
  'name': 'Helio',
  'offices': {'address1': '10960 Wilshire Blvd.'},
  'coord': {'type': 'Point', 'coordinates': [-118.446596, 34.057498]}},
 {'_id': ObjectId('5fb00ed5a020aa91fd925202'),
  'name': 'Stickam',
  'offices': {'address1': '1975 pads rd'},
  'coord': {'type': 'Point', 'coordinates': [-118.254558, 34.051409]}},
 {'_id': ObjectId('5fb00ed5a020aa91fd925227'),
  'name': 'TripUp',
  'offices': {'address1': None},
  'coord': {'type': 'Point', 'coordinates': [-118.243425, 34.052187]}},
 {'_id': ObjectId('5fb00ed5a020aa91fd92528c'),
  'name': 'ScanScout',
  'offices': {'address1': '11601 Wilshire Blvd'},
  'coord': {'type': 'Point', 'coordinates': [-118.459995, 34.050887]}},
 {'_id': ObjectId('5fb00ed5a020aa91fd9252d4'),
  'name': 'WebbAlert',
  'offices': {'address1': None},
  'coord': {'type': 'Point', 'coordinates': [-118.243425, 34.052187]}},
 {'_id': ObjectId('5fb00ed5a020aa91fd9252f8'),
  'name': 'Revver',
  'offices': {'address

In [6]:
name = []
latlng = []
latlng_str = []
for i in range(166):
    name.append(res[i]['name'])
    latlng.append(res[i]['coord']['coordinates'][::-1])

In [15]:
df = pd.DataFrame()
df['name'] = name
df['office_lst'] = latlng
for lst in latlng:
    latlng_str.append(f"{lst[0]},{lst[1]}")
df['office_str'] = latlng_str
df

,name,office_lst,office_str
0,Helio,"[34.057498, -118.446596]","34.057498,-118.446596"
1,Stickam,"[34.051409, -118.254558]","34.051409,-118.254558"
2,TripUp,"[34.052187, -118.243425]","34.052187,-118.243425"
3,ScanScout,"[34.050887, -118.459995]","34.050887,-118.459995"
4,WebbAlert,"[34.052187, -118.243425]","34.052187,-118.243425"
...,...,...,...
161,AECOM Technology,"[34.0507962, -118.2568615]","34.0507962,-118.2568615"
162,Culture Jam,"[34.0137151, -118.4660597]","34.0137151,-118.4660597"
163,Pipit Interactive,"[34.045499, -118.47036]","34.045499,-118.47036"
164,Top Rated Website Designers,"[33.9933257, -118.3987842]","33.9933257,-118.3987842"


In [18]:
df.to_csv('offices_LA.csv')